# Airfoil Learning Tutorial
In this tutorial you will learn how to run the code provided to setup the code for training. 
Data Processing
1. Downloading the Dataset
2. Normalization
3. Creating the Graph Network Database (This can take time)
4. Creating Multilayer Perception Database (Done with step 3)

Training 
1. Data fractionalizing - The entire dataset is large and can take a long time to train. This step takes a fraction of the data and uses it for training
2. Train Graph Network
3. Train Multilayer Perception

**Outcome of this tutorial**
When you look through the code, you'll be more familar with how it works. What file does what. This tutorial is a walk through of the code cited in the readme.md inside of `generate_xfoil` and `pytorch` folders. 




# Background
The section below describes some of the theory behind Lift, Drag, Moment, and Pressure. You don't need to understand everything in detail but if you wanted to, there's good references. 

**To the students**: feel free to file a github issue with documentation if something is unclear or you have questions. 

**To the instructors**: if something is unclear, please help make this documentation better by making changes and doing a pull request. 


The image below is a airfoil. Airfoils are a 2D cross section of an airplane wing. These are some of the parameters used to characterize an airfoil.

<p align="center">
  <img src="https://www.researchgate.net/profile/Adson-De-Paula/publication/305044784/figure/fig22/AS:669442585399311@1536618958931/Main-geometric-parameters-of-an-aerodynamic-airfoil.png" alt="airfoil"/>
</p>



## Performance Characterization
Airfoil performance are characterized by normalized Lift, Drag, and Moment coefficients. Think of Lift as a force pushing the airfoil up in the presence of air flow. In order to have Lift, you need to have Drag; however, not all airfoils have the same efficiency, some produce more lift with less drag than others at certain conditions.

<p align="center">
  <img src="https://i.stack.imgur.com/MlDJx.png" alt="airfoil free body diagram"/>
</p>




### Lift
To understand what Lift is, you need to understand pressure forces. These are upward and downward forces distributed along the airfoil. When you integrate the **vertical** component of the pressure force $p(x)$ there's a certain location where everything equals to 0, this is called the center of pressure. In a freebody diagram, this is the balancing point where lift is acting on. 

<p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/Airfoils_-_pressure_diagrams.svg/500px-Airfoils_-_pressure_diagrams.svg.png" alt="airfoil lift distribution"/>
</p>

\begin{align}
x_{cp} = \int_{x_{LE}}^{x_{TE}} \frac{x p(x)}{p(x)}
\end{align}

The coefficient of lift is simply the total lift force (L) normalized by the dynamic pressure $q=0.5 \rho V^2$

\begin{align}
C_L = \frac{L}{q}
\end{align}


### Moment

Moment coefficient or pitching moment coefficient affects how the airfoil want to pitch based on airflow and angle of attack of which the air is impacting the airfoil. Moment acts on the aerodytnamic center $x_{ac}$. This is the point where all moment forces are 0. 


The equation is the moment which is equal to $M=L*d$ where $d$ is the distance from the center of pressure to the aerodynamic center (Point at which moment forces are 0)

\begin{align}
C_{M} = \frac{M}{Sc 0.5\rho V^2}
\end{align}

To compute the aerodynamic center $x_{ac}$ you need to know how the coefficient of moment changes with angle of attack $\frac{dCm}{d\alpha}$ and how the coefficient of lift changes with angle of attack $\frac{dCl}{d\alpha}$. These comes from simulations with [XFOIL](https://web.mit.edu/drela/Public/web/xfoil/) (Note: XFLR is running xfoil in the background) or a Computational Fluid Dynamics (CFD) Tool. Xfoil uses $0.25*c$ or 25% of the chord as a reference x-value in computing the coefficient of moment. 

What we are computing below is the aerodynamic center of the wing. This is where the moment is 0. For airfoil stability, the aerodynamic center should be in front of the center of gravity - the airfoil will want to pitch upwards as opposed to downwards toward the earth. 

\begin{align}
x_{ac} = -\frac{ \frac{dC_M}{d\alpha} }{ \frac{dCl}{d\alpha} }+ 0.25
\end{align}


> Good reference for Lift-Drag-Moment Coefficient https://aerotoolbox.com/lift-drag-moment-coefficient/

> XFoil Documentation
> See: http://web.mit.edu/aeroutil_v1.0/xfoil_doc.txt

### Drag

Airfoil drag is a combination of **viscous drag forces** and **pressure drag forces**.

#### Viscous Drag
Viscous drag is due to shear forces boundary layer. A fluid particle at the surface of a body has 0 velocity, but as you go further from the surface, the velocity is equal to the free stream. This region is the boundary layer. The boundary layer can be laminar, transitional, or turbulent. In simple airfoil theory, e^N transition model is used to determine the state of the boundary layer 

> Laminar boundary layer
> https://en.wikipedia.org/wiki/Blasius_boundary_layer

> $e^N$ Transition Criteria
> https://arc.aiaa.org/doi/10.2514/6.2003-4066

Understanding the state of the boundary layer is important for the next step which is to compute the height of the boundary layer. The height is used to compute momentum thickness which is then used to compute the drag. 

> Emperical correlations for boundary layer thickness https://en.wikipedia.org/wiki/Boundary_layer_thickness

Using equation 19.19 from http://www.ase.uc.edu/class/AEEM456/Section_3_Notes.pdf and correlations, it's possible to estimate the shear stress which is the viscous stress. 

#### Pressure Drag 
Pressure drag is caused by the pressure forces acting **axially** $P\hat{x}$ on the airfoil. Pressure is alway normal to the surface so if the surface is at an inclination you can break down pressure into two components a vertical component $P\hat{y}$ and a horizontal/axial component $P\hat{x}$. $P\hat{y}$ is used for calculating lift and $P\hat{x}$ for drag. This is the simple explaination. This is what panel code uses to compute pressure drag.

<p align="center">
  <img src="https://github.com/nasa/airfoil-learning/blob/main/images/tutorial-pressure.jpg?raw=true" alt="Pressure breakdown into components"/>
</p>


## Cloning the Project
This section shows how to create a dataset for training purposes

In [ ]:
!git clone https://github.com/nasa/airfoil-learning.git
!cp -r airfoil-learning/generate_xfoil/* .

Cloning into 'airfoil-learning'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 71 (delta 13), reused 59 (delta 6), pack-reused 0
Unpacking objects: 100% (71/71), done.


# Data Processing
To begin data processing, the notebook structure needs to be reorganized. This is not needed when dealing with the actual code on your computer. Jupyter notebook does not allow you to change root directories so we have to copy everything to the root and process it then move it back. 


## Downloading the Dataset
Download the dataset by running the code below. Contents of airfoil-learning/generate_xfoil are copied to root path. This is because you cannot change your working directory from within a jupyter notebook

In [ ]:
!rm -rf json/
!wget https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/airfoil-learning-dataset.zip
!unzip airfoil-learning-dataset.zip
!rm airfoil-learning-dataset.zip


--2022-03-23 14:40:18--  https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/airfoil-learning-dataset.zip
Resolving nasa-public-data.s3.amazonaws.com (nasa-public-data.s3.amazonaws.com)... 52.216.204.163
Connecting to nasa-public-data.s3.amazonaws.com (nasa-public-data.s3.amazonaws.com)|52.216.204.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1569105133 (1.5G) [application/zip]
Saving to: ‘airfoil-learning-dataset.zip’

airfoil-learning-da 100%[===================>]   1.46G  54.5MB/s    in 29s     

2022-03-23 14:40:47 (52.0 MB/s) - ‘airfoil-learning-dataset.zip’ saved [1569105133/1569105133]

Archive:  airfoil-learning-dataset.zip
  inflating: json/2032c-il - 20-32C AIRFOIL.json  
  inflating: json/a18-il - A18 (original).json  
  inflating: json/a63a108c-il - NASA_AMES 63A108 MOD C AIRFOIL.json  
  inflating: json/ag04-il - AG04.json  
  inflating: json/ag09-il - AG09.json  
  inflating: json/ag11-il - AG11.json  
  inflating: json/ag13-il - AG1

### Install pytorch geometric 


In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu115.html
!pip3 install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html


Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu115.html
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


## Normalization

Normalization brings all the data to the same scale. Without this, the weight matrix may be illconditioned and you'll have extremely high gradients. 

ResizeCp.py - this file takes all the json data and resizes the Cp to 50 points on pressure side and 50 points on suction side - the JSON contains about 100 points on both sides describing Cp. 

Step3_NormalizeData.py takes into account the entire dataset and normalizes each of the fields. So if airfoil has 50 points in the x and y direction to define the pressure side. Point 1 y-value for all airfoils taken as a vector and normalized between 0 and 1 or mean and standard deviation same with point 2 all the way to the last point. This is an example of local normalization. You can select to normalize by the locally or by the global value. Results can vary and consistency is important when comparing graph and multilayer perception/deep neural networks. 

The output for all this is a file called `scalers.pickle`


In [ ]:
!python ResizeCp.py
!python Step3_NormalizeData.py

Processing: 100% 825/825 [13:50<00:00,  1.01s/it]
reading json data: 100% 825/825 [00:09<00:00, 85.45it/s]


## Creating the Processed Dataset for Graph and Multilayer Perception Networks
Next step is to iterate through the JSON files, normalize, and create pytorch geometric dataset for the training step. 


Option 1: Run the code below to generate test and train dataset. **This may take a while ...**

In [ ]:
!python Step4_CreateDataset.py

Processing:  28% 234/825 [24:16<2:51:46, 17.44s/it]^C


Option 2: Download the processed dataset. This extracts to a `datasets` folder. There's the `datasets/minmax` this is the data scaled using minmax scaler. `datasets/standard` is the data scaled using standard scaler. 

> Note: This may not work for newer versions of pytorch or pytorch geometric . 

In [ ]:
!wget https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/dataset-processed.zip
!unzip dataset-processed.zip


--2022-03-23 15:15:44--  https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/dataset-processed.zip
Resolving nasa-public-data.s3.amazonaws.com (nasa-public-data.s3.amazonaws.com)... 52.216.77.4
Connecting to nasa-public-data.s3.amazonaws.com (nasa-public-data.s3.amazonaws.com)|52.216.77.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9188863364 (8.6G) [application/zip]
Saving to: ‘dataset-processed.zip’

dataset-processed.z 100%[===================>]   8.56G  55.4MB/s    in 2m 58s  

2022-03-23 15:18:42 (49.3 MB/s) - ‘dataset-processed.zip’ saved [9188863364/9188863364]

Archive:  dataset-processed.zip
   creating: datasets/
   creating: datasets/minmax/
  inflating: datasets/minmax/dnn_scaled_data_cp_test.pt  
  inflating: datasets/minmax/dnn_scaled_data_cp_train.pt  
  inflating: datasets/minmax/dnn_scaled_data_test.pt  
  inflating: datasets/minmax/dnn_scaled_data_train.pt  
  inflating: datasets/minmax/graph_scaled_data_cp_test.pt  
  inflating: d

In [ ]:
# Reorganize
!mv datasets airfoil-learning/generate_xfoil/
!mv json* airfoil-learning/generate_xfoil/
!mv libs* airfoil-learning/generate_xfoil/
!mv old* airfoil-learning/generate_xfoil/
!mv xfoil* airfoil-learning/generate_xfoil/
!mv *.py airfoil-learning/generate_xfoil/
!mv *.pickle airfoil-learning/generate_xfoil/

!rm *.zip
!rm -r libs old sample_data xfoil

mv: cannot stat 'datasets': No such file or directory
mv: cannot stat 'json*': No such file or directory
mv: cannot stat 'libs*': No such file or directory
mv: cannot stat 'old*': No such file or directory
mv: cannot stat 'xfoil*': No such file or directory
mv: cannot stat '*.py': No such file or directory
mv: cannot move 'scalers.pickle' to 'airfoil-learning/generate_xfoil/': Not a directory
rm: cannot remove '*.zip': No such file or directory
rm: cannot remove 'libs': No such file or directory
rm: cannot remove 'old': No such file or directory
rm: cannot remove 'sample_data': No such file or directory
rm: cannot remove 'xfoil': No such file or directory


# Training

To begin training, the notebook structure needs to be reorganized. This is not needed when dealing with the actual code on your computer. Jupyter notebook does not allow you to change root directories so we copy everything to the root and process it then move it back. 

To begin training, we need to setup or proportion the dataset. The full processed and normalized dataset is located in `../generate_xfoil`. For training we can either train on the entire thing or on a small fraction of the data.


In [ ]:
!mv airfoil-learning/generate_xfoil .
!mv airfoil-learning/pytorch .
!rm *.png

In [ ]:
# Changes the directory to pytorch and executes setup_dataset.py
# What you would do in vscode is open `pytorch` as your working directory 
#    and then execute setup_dataset.py from there 
!(cd pytorch && python setup_dataset.py > setup_dataset.out)

# Note: Depending on the speed and if you paid for colab pro, this command may die unexpectedly. 

If the above block of code doesn't work then run the code below. This is a modified version of setup_dataset.py.

In [ ]:
from pathlib import Path
from typing import List
import torch
import json
from sklearn.model_selection import train_test_split 
import json
import os.path as osp

def setup_dataset(train_filename:str,test_filename:str,percent_dataset:float=1,train_percentage:float=0.8,train_indices:List[int]=None, test_indices:List[int]=None,scaler_type:str="minmax"):
    """Setup the Train and Validation datasets

    Args:
        train_filename (str): [description]
        test_filename (str): [description]
        percent_dataset (float, optional): percentage of dataset to use 0 to 1. Defaults to 1.
        train_indices (List[int], optional): array indices to use for training. Defaults to None.
        test_indices (List[int], optional): array indices to use for test. Defaults to None.
        scaler_type (str, optional): string to describe the type of scaler to use. This determines where the dataset is saved. Defaults to "minmax".

    Returns:
        (tuple): tuple containing:

            - **train_dataset** (torch.utils.data.TensorDataset): pitch distribution
            - **val_dataset** (torch.utils.data.TensorDataset): pitch to chord distribution

    """

    train_dataset = torch.load(train_filename)
    test_dataset = torch.load(test_filename)
    
    # Combined train and test
    train_dataset.extend(test_dataset)
    
    if (not train_indices):
        all_data = range(len(train_dataset))
        # Shuffle the dataset and select a percent from it 
        new_dataset,_ = train_test_split(all_data, test_size=1-percent_dataset, train_size=percent_dataset)
        train_indices, test_indices = train_test_split(new_dataset, test_size=1-train_percentage, train_size=train_percentage)
    
    test_dataset = [train_dataset[t] for t in test_indices]
    train_dataset = [train_dataset[t] for t in train_indices]
    
    Path(osp.join('local_dataset',scaler_type)).mkdir(parents=True, exist_ok=True)
    Path(osp.join('local_dataset',scaler_type)).mkdir(parents=True, exist_ok=True)
    if 'graph' in train_filename:
        data_params = {'input_size':train_dataset[0].x.shape[0],'output_size':len(train_dataset[0].y),'node_labels':train_dataset[0].node_labels.shape[0]}
        with open(osp.join('local_dataset',scaler_type,'gnn_dataset_properties.json'), 'w') as outfile:
            json.dump(data_params, outfile)
        torch.save(train_dataset,osp.join('local_dataset',scaler_type,'train_gnn.pt'))
        torch.save(test_dataset,osp.join('local_dataset',scaler_type,'test_gnn.pt'))
    else:
        features = train_dataset[0][0]
        labels = train_dataset[0][1]
        data_params = {'input_size':len(features),'output_size':len(labels)}
        with open(osp.join('local_dataset',scaler_type,'dnn_dataset_properties.json'), 'w') as outfile:
            json.dump(data_params, outfile)
        torch.save(train_dataset,osp.join('local_dataset',scaler_type,'train_dnn.pt'))
        torch.save(test_dataset,osp.join('local_dataset',scaler_type,'test_dnn.pt'))
    
    return train_indices, test_indices

if __name__ == '__main__':
    with open('settings.json', 'r') as infile:    # Loads the settings for all networks
        settings = json.load(infile)
    
    percent_train = -1 
    for args in settings['data']:
        if percent_train == args["percent_train"]:
            train_indices, test_indices = setup_dataset(args['train_filename'].replace("../",""), args['test_filename'].replace("../",""), args['percent_dataset'], args["percent_train"],train_indices,test_indices,args['scaler_type'])
        else:
            train_indices, test_indices = setup_dataset(args['train_filename'].replace("../",""), args['test_filename'].replace("../",""), args['percent_dataset'], args["percent_train"],None,None,args['scaler_type'])
        percent_train = args["percent_train"]
    print('local_dataset created')

If the above 2 codes do not work and you notice session crashing because of not enough ram or disk then you need to pay for colab pro or build/find a better computer. I ran this on a machine with 64GB ram and had no issues.

## Multilayer Perception Networks

 A multilayer perception network is a network that is made up of fully connected linear layers (pytorch) or dense layers (tensorflow). 

![mlp](https://raw.githubusercontent.com/nasa/airfoil-learning/main/images/mlp.png)

Here is an example of how to code up a multilayer linear network from the file `MultiLayerLinear.py`. The way to initialize is it:  

```python
model = MultiLayerLinear(3,4,[16,16,16,256,560,5])
```
The array indicates the sizes of the hidden layers. 3 is the number of inputs and 4 is number of outputs.


```python
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Module, Dropout
from typing import List
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class MultiLayerLinear(Module):
    """
    Model aims to predict both cp and y with spatial convolutions (MFConv).
    """
    # Multi-layer Perceptron 
    # https://discuss.pytorch.org/t/how-to-create-mlp-model-with-arbitrary-number-of-hidden-layers/13124/2

    # https://github.com/FrancescoSaverioZuppichini/Pytorch-how-and-when-to-use-Module-Sequential-ModuleList-and-ModuleDict

    def __init__(self,in_channels:int,out_channels:int,h_sizes:List[int]=None):
        """This class joins a bunch of linear layers together to predict the output size

        Args:
            in_channels (int): number of inputs channels
            out_channels (int): number of output channels
            h_sizes (List[int], optional): Any additional internal linear layers. Defaults to None.
        """        
        super(MultiLayerLinear, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.layers = nn.ModuleList()
        if (h_sizes!=None):
            self.layers.append(nn.Linear(in_channels,h_sizes[0],bias=True))
            self.layer_sizes = h_sizes
            for k in range(len(h_sizes)-1):
                self.layers.append(nn.Linear(h_sizes[k], h_sizes[k+1],bias=True))
            self.layers.append(nn.Linear(h_sizes[len(h_sizes)-1], out_channels,bias=True))
        else:
            self.layers.append(nn.Linear(in_channels,out_channels))
            self.layer_sizes = None
   
    def forward(self,x):
        out = x
        for i in range(len(self.layers)-1):
            out = F.relu6(self.layers[i](out))        
        return self.layers[-1](out) # dont activate the last laye

    def __str__(self):
        n_layers = len(self.layers)
        if (self.layer_sizes):
            layer_sizes = '-'.join([str(x) for x in self.layer_sizes])
            return f"MLL_IN-{self.in_channels}_OUT-{self.out_channels}_{layer_sizes}"
        else:
            return f"MLL_IN-{self.in_channels}_OUT-{self.out_channels}"

    def __repr__(self):
        return self.__str__()

```

In [ ]:
!(cd pytorch && python train_dnn.py)

# Graph Neural Network
The graph network used was [spline convolution](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/conv/spline_conv.html). This type of graph convolution was used in an encoder-decoder style network to learn features. 

![encoder-decoder](https://github.com/nasa/airfoil-learning/blob/main/images/encoder-decoder.png?raw=true)

In an encoder-decoder network each feature encoded is saved and passed directly into the decoder. Here's the example from `pytorch/gnn_model.py`

In the initialization `__init__`, I had to initialize the size of the decoder to take into account the size of the features coming out of the encoder as well as the features entering in the decoder from past decoders. 

Example: 
```python
  for n in range(len(neurons)-1,0,-1):
      out_channels = self.encoder[n_encoder-i].out_channels 
      self.decoder.append(SplineConv(out_channels+neurons[n], neurons[n], dim=2,degree=2,kernel_size=3))
      self.decoder_bn.append(BatchNorm(neurons[n]))
      i+=1
```

In [ ]:
import torch
from typing import List
import torch.nn.functional as F
from torch.nn import Linear, Module, Dropout, ModuleList
from torch_geometric.data import Data
from torch_geometric.nn import BatchNorm
# from torch_geometric.nn import SplineConv
from SplineConv import SplineConv
from MultiLayerLinear import MultiLayerLinear

class GnnModel(Module):
    """Defines the graph neural network structure for prediction 
        Example of encoder decoder with cnn and batchnorms https://medium.com/dataseries/convolutional-autoencoder-in-pytorch-on-mnist-dataset-d65145c132ac
    """
    def __init__(self,linear_input_size:int, neurons:List[int]=[16,32,64],linear_layers:MultiLayerLinear = None,batch_norm_encoder=False, batch_norm_decoder=False):
        """Constructor for GnnModel. User passes the number of neurons they would like the graph network to use. The graph network then constructs and encoder with those neurons and corresponding decoder. 

        Args:
            linear_input_size (int): 
            neurons (List[int], optional): Number of neurons to use for each graph neural network. Defaults to [16,32,64].
            use_batch_norm (bool, optional): Whether to use batch norm in between encoder networks. Defaults to False.
            dropout (float, optional): [description]. Defaults to 0.
            linear_layers (MultiLayerLinear, optional): [description]. Defaults to None.
        """
        super(GnnModel, self).__init__()
        self.linear_input_size = linear_input_size
        self.batch_norm_encoder = batch_norm_encoder
        self.batch_norm_decoder = batch_norm_decoder
        self.encoder = ModuleList()
        self.encoder_bn = ModuleList()
        self.neurons = neurons
        # Encoder 
        for n in range(1,len(neurons)): 
            self.encoder.append(SplineConv(neurons[n-1], neurons[n], dim=2,degree=2,kernel_size=3))
            self.encoder_bn.append(BatchNorm(neurons[n]))
        self.encoder.append(SplineConv(neurons[n], neurons[n], dim=2,degree=2,kernel_size=3))
        self.encoder_bn.append(BatchNorm(neurons[n]))

        # Decoder 
        self.decoder = ModuleList()
        self.decoder_bn = ModuleList()
        i = 0
        n_encoder = len(self.encoder)-1
        for n in range(len(neurons)-1,0,-1):
            out_channels = self.encoder[n_encoder-i].out_channels 
            self.decoder.append(SplineConv(out_channels+neurons[n], neurons[n], dim=2,degree=2,kernel_size=3))
            self.decoder_bn.append(BatchNorm(neurons[n]))
            i+=1
        self.linear_layers = linear_layers
    
    def forward(self,data:Data):
        """Sends torch geometric data through the neural network. 

        Args:   
            data (torch_geometric.data.Data): https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html 

        Returns:
            (torch.tensor): tensor describing the results of the network. This depends on what is predicted by MultiLayerLinear
        """
        x, edge_index, edge_attr, batch, pos = data.x,data.edge_index, data.edge_attr, data.batch, data.pos
        batch_size = max(batch)+1
        # Run through encoder and store results 
        out = list()
        out.append(x)
        for encoder in self.encoder:
            out.append(F.relu6(encoder(out[-1],edge_index,edge_attr=pos)))
        # Run through decoder and apply results from encoder 
        n = len(out)-3
        dout_out = torch.cat([out[-1], out[-2]],dim=1) 
        for decoder in self.decoder:
            dout = F.relu6(decoder(dout_out, edge_index, edge_attr=pos))     # https://arxiv.org/pdf/1711.08920.pdf From SplineConv Paper G = (V, E, U) where U is the position and also edge attributes
            dout_out = torch.cat([dout, out[n]],dim=1)                            # data = spline_basis(edge_attr, self.kernel_size, self.is_open_spline, self.degree) Spline basis
            n -= 1

        # conditions = conditions.reshape((batch_size,3))
        out = dout.reshape((batch_size,self.linear_input_size*dout.shape[1]))
        # out8 = torch.cat((out7,conditions),dim=1)
        return self.linear_layers(out)
    
    def __str__(self):
        n_encoders = len(self.encoder)
        n_decoders = len(self.decoder)
        neurons = '-'.join([str(n) for n in self.neurons])
        linear = str(self.linear_layers)
        return f"GnnModel_{neurons}_{n_encoders}_{n_decoders}_{str(self.batch_norm_encoder)}_{str(self.batch_norm_decoder)}_MLinear_{linear}"

To train the network you can run the code below 


> Note: Training will certainly not work on colab without lots of $$ thrown at google.


In [ ]:
!(cd pytorch && python train_gnn.py)     # Trains the network to predict only Cl,Cd,Cdp,Cm
!(cd pytorch && python train_gnn_cp.py)  # Trains the network to predict Cl,Cd,Cdp,Cm, and Cp all together

# Plotting and Visualizing the Results

Once training is complete, you'll have a bunch of model weights in the checkpoints folder. Keeping all this information is useful for going back and evaluating each model. 

To plot a randomly selected design, simply run 

In [ ]:
!(cd pytorch python plot_random_airfoil_results.py)

IF you want to visualize the training results. You can run 

In [ ]:
!(cd pytorch python plot_train_history.py)

The training saves the loss for train and test inside of the checkpoints. It's important to keep everything including the model weights. 

I was not able to run this in google colab. I reccomend running all the code locally or within some kind of super computer. 